In [1]:
!pip install -q -U transformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.9 MB/s eta 0:00:00


In [2]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/sem 3 code/GSM_Symbolic_Robustness-main/dataset

/content/drive/.shortcut-targets-by-id/1PFSA5OH_-TKMa-zkUK7ycyKv-lMEbUgr/sem 3 code/GSM_Symbolic_Robustness-main/dataset


In [ ]:
# !unzip GSM_Symbolic_Robustness-main.zip

In [ ]:
# %cd /content/GSM_Symbolic_Robustness/dataset

[Errno 2] No such file or directory: '/content/GSM_Symbolic_Robustness/dataset'
/content


In [ ]:
config = {
    'model_name': "microsoft/Phi-3.5-mini-instruct",
    'tokenizer_name': 'microsoft/Phi-3.5-mini-instruct',
    'token': 'ADD HUGGING FACE TOKEN HERE',
    'is_mistral_or_llama': False,
    'output_name': 'test_output',
    'val_dataset_path': 'gsm_symbolic/data/noop_v1.jsonl',
    'use_calculator': True,
    'max_calculator_regenerations': 3,
    'finetune': False,
    'fine_tuned_model_path': 'fine_tuned_phi.pt',
    'use_noop_engineered_prompt': False
}

In [ ]:
# import importlib
# importlib.reload(dataset_and_dataloader)

In [6]:
import dataset_and_dataloader
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM
import torch
import re
import numpy as np
from tqdm import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['tokenizer_name'], token = config['token'])
if config['is_mistral_or_llama']:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
val_dataset = dataset_and_dataloader.GSM8K_Val_Dataset(config['val_dataset_path'], tokenizer, use_noop_prompt=config['use_noop_engineered_prompt'])
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [13]:
# load in quantization config
q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = config["model_name"]

model = AutoModelForCausalLM.from_pretrained(model_name, token=config['token'], quantization_config=q_config)
if config['is_mistral_or_llama']:
  model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [14]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [23]:
#free CUDA memory
torch.cuda.empty_cache()
import gc
gc.collect()

20671

In [ ]:
if config['finetune']:
  state_dict = torch.load(config['fine_tuned_model_path'])
  state_dict = {k[len("base_model.model.model."):] if k.startswith("base_model.model.model.") else k: v for k, v in state_dict.items()}
  model.load_state_dict(state_dict, strict=False)

In [ ]:
model.to(device)
model.eval()

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm):

In [ ]:
def extract_a(text):
  regex = re.compile(r"#### (\-?[0-9\.\,]+)")
  model_answer = regex.search(text)
  if model_answer:
    answer = model_answer.group(1)
    answer = answer.replace(",", "")
    answer = answer.replace(" ", "")
    return answer
  else:
    return "No Valid Answer Found"

def get_accuracy(outputs, a):
  pred_a = [extract_a(out) for out in outputs]
  a = [extract_a(ans) for ans in a]
  error_rate = np.mean(np.array(pred_a) == "No Valid Answer Found")
  return np.mean(np.array(pred_a) == np.array(a)), error_rate

In [ ]:
calculator_regex = re.compile(r"<<([0-9\.\, \*\+\-=/\(\)]*)>>")

In [ ]:
acc = 0
error_rate = 0
pbar = tqdm(val_dataloader)
num_batches_done = 0
to_write = dict()
for q_id, q, q_mask, a in pbar:
    # move to device
    q = torch.stack(q, dim=1).to(device)
    q_mask = torch.stack(q_mask, dim=1).to(device)
    # do inference
    pred_a = model.generate(
      input_ids=q,
      attention_mask=q_mask,
      max_new_tokens=256
    )
    outputs = tokenizer.batch_decode(pred_a[:, q.shape[-1]:], skip_special_tokens=True)

    if config['use_calculator']:
      # CALCULATOR!
      error_counter = 0
      i = 0
      while i < len(outputs) and error_counter < config['max_calculator_regenerations']:
        if "<<" in outputs[i]:
          check_output = calculator_regex.findall(outputs[i])
          for equation in check_output:
            equation = equation.replace("=", "==")
            if not eval(equation):
              print("REGENERATING: wrong equation -> "+ equation)
              pred_a = model.generate(
                input_ids=q,
                attention_mask=q_mask,
                max_new_tokens=256
              )
              outputs = tokenizer.batch_decode(pred_a[:, q.shape[-1]:], skip_special_tokens=True)
              error_counter += 1
              i = -1
              break
        i += 1

    to_write.update(dict(zip(q_id, outputs)))
    new_acc, new_error_rate = get_accuracy(outputs, a)
    acc += new_acc
    error_rate += new_error_rate
    num_batches_done += 1
    # do display
    pbar.set_postfix(accuracy=f"{(acc/num_batches_done):.2f}%", loss=f"{(error_rate/num_batches_done):.2f}")
    pbar.update(1)
acc /= len(val_dataloader)
error_rate /= len(val_dataloader)
print(f"\nAccuracy: {acc}")
print(f"Error Rate: {error_rate}")

  1%|          | 1/100 [00:13<22:10, 13.44s/it, accuracy=0.00%, loss=0.00]

REGENERATING
REGENERATING
REGENERATING


  2%|▏         | 2/100 [01:09<56:29, 34.59s/it, accuracy=0.00%, loss=0.00]


KeyboardInterrupt: 

In [ ]:
with open("outputs/"+config['output_name']+"_acc="+str(acc)+"_error="+str(error_rate)+".txt", "w") as f:
  f.write(str(to_write))